In [ ]:
import os
import sys
from pathlib import Path

ROOT_DIR = Path(os.getcwd()).parent.parent
sys.path.append(str(ROOT_DIR))

In [ ]:
import logging
from datetime import date

import polars as pl
from stable_baselines3 import TD3

from models.DPT import DPTEnv, TensorboardCallBack
from utils.polars import TimesSeriesPolars

logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s', force=True)

In [ ]:
data = pl.read_csv( ROOT_DIR / "data/index_historical/SP500_ohlcv_2004-11-30_to_2024-12-31.csv", schema_overrides={"Date": date})
data_holder = TimesSeriesPolars(data=data, index_ticker='GSPC.INDX')
data_holder.calculate_logR()
full_returns = data_holder.get('logR', include_index=True)

In [ ]:
env = DPTEnv(full_returns, index_ticker='GSPC.INDX')
env.reset()
model = TD3('MultiInputPolicy', env, verbose=1)

In [ ]:
# from stable_baselines3.common.env_checker import check_env
# check_env(env)

In [ ]:
# TIMESTEPS = 100
# for _ in range(5):
#     model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name='TD3', callback=TensorboardCallBack())


In [ ]:
episodes = 2
for episode in range(1, episodes+1):
    obs = env.reset()
    terminated = False
    score = 0
    env.render()
    while not terminated:
        action = env.action_space.sample()
        n_state, reward, terminated, truncated, info = env.step(action)
        if info['retry_count'] == 0:
            env.render()
        score += reward
    print(f'Episode: {episode} - Score:{score}')

In [1]:
import numpy as np